# 형태소 분석 기반 토큰화의 문제

-   형태소 분석기는 작성된 알고리즘 또는 학습된 내용을 바탕으로 토큰화를 하기 때문에 오탈자나 띄어쓰기 실수, 신조어, 외래어, 고유어 등이 사용된 경우 제대로 토큰화 하지 못한다.
-   그래서 발생 할 수있는 잠재적 문제점 - 어휘사전을 크게 만든다. - 같은 의미의 단어가 형태소 분석이 안되어 여러개 등록될 수있다. - ex) 신조어 `돈쭐` 이라는 단어를 인식 못할 경우 `"돈쭐내러", "돈쭐나", "돈쭐냄"` 등이 다 등록 될 수 있다. - OOV(Out Of Vocab)에 대응하기 어렵게 만든다. - 같은 어근의 단어가 있지만 조사등이 바뀐 신조어등을 OOV로 인식할 수있다.
    > -   Out Of Vocab(OOV)
    >     -   Vocab(어휘사전): 코퍼스를 구성하는 토큰들의 모음이다.
    >     -   OOV는 어휘사전에 없는 토큰을 말한다.

# Subword Tokenization(하위 단어 토큰화)

## 개념

-   하나의 단어를 작은 단위의 하위단어(subword)들의 조합으로 나누는 방식으로 토큰화 한다.
    -   자주 쓰이는 문자조합은 더 작은 subword로 분리하면 안된다.
    -   희귀한 문자조합은 의미있는 더 작은 subword로 분리되어야 한다
    -   ex) normally 가 희귀 단어로 분류되면 'normal'과 'ly' 로 분리한다.

## 잇점

-   token 단어의 길이를 줄일 수 있어 처리 속도가 빨라진다.
-   OOV 문제, 신조어, 은어, 고유어등의 문제를 완화할 수 있다.

## 방법

-   Byte Pair Encoding (BEP), Word Piece, Unigram Model 등


# Byte Pair Encoding

-   원래 Text data 압축을 위해 만들어진 방법으로 text 에서 많이 등장하는 두글자 쌍의 조합을 찾아 부호화하는 알고리즘이다.
-   연속된 글자 쌍이 더 나타나지 않거나 정해진 어휘사전 크기에 도달 할 때 까지 조합을 찾아 부호화 하는 작업을 반복한다.

## text 압축 방식의 예

-   원문: abracadabra

1. AracadAra: ab -> A :=> 원문에서 가장 빈도수 많은 ab를 A(부호로 아무 글자나 사용할 수 있다.)로 치환
2. ABcadAB: ra -> B :=> 1에서 가장 빈도수가 많은 ra를 B로 치환
3. CcadC: AB -> C :=> 2에서 가장 빈도수 맣은 AB를 C로 치환한다.(치환된 글자 쌍도 변환대상에 포함된다.)

## BPE Tokenizer 방식

BPE 토크나이저는 자주 등장하는 글자 쌍을 찾아 치환하는 대신 **단어 사전**에 추가한다.

### 예)

1. 말뭉치의 토큰들의 빈도수, 어휘사전은 아래와 같을 경우
    - 빈도사전: ('low', 5), ('lower', 2), ('newest', 6), ('widest', 3)
    - 어휘사전: ['low', 'lower', 'newest', 'widest']
2. 빈도 사전내의 모든 단어들을 글자 단위로 나눈다.
    - 빈도사전: ('l', 'o', 'w', 5), ('l', 'o', 'w', 'e', 'r', 2), ('n', 'e', 'w', 'e', 's', 't', 6), ('w', 'i', 'd', 'e', 's', 't', 3)
    - 어휘사전: ['d', 'e', 'i', 'l', 'n', 'o', 'r', 's', 't', 'w']
3. 빈도 사전을 기준으로 가장 자주 등장하는 글자 쌍(byte pair)를 찾는다. 위에서는 **'e'와 's'가 총 9번으로 가장 많이 등장함**. 'e'와 's'를 'es'로 합치고 어휘 사전에 추가한다.
    - 빈도사전: ('l', 'o', 'w', 5), ('l', 'o', 'w', 'e', 'r', 2), ('n', 'e', 'w', **'es'**, 't', 6), ('w', 'i', 'd', **'es'**, 't', 3)
    - 어휘사전: ['d', 'e', 'i', 'l', 'n', 'o', 'r', 's', 't', 'w', **'es'**]
4. 3 번의 과정을 계속 반복한다. 빈도수가 가장 많은 'es'와 't' 쌍을 'est'로 병합하고 'est'를 어휘 사전에 추가한다.
    - 빈도사전: ('l', 'o', 'w', 5), ('l', 'o', 'w', 'e', 'r', 2), ('n', 'e', 'w', **'est'**, 6), ('w', 'i', 'd', **'est'**, 3)
    - 어휘사전: ['d', 'e', 'i', 'l', 'n', 'o', 'r', 's', 't', 'w', **'es'**, **'est'**]
5. 만약 10번 반복했다고 하면 다음과 같은 빈도 사전과 어휘 사전이 생성된다.
    - 빈도 사전: (**'low'**, 5), (**'low'**, 'e', 'r', 2), ('n', 'e', 'w', **'est'**, 6), ('w', 'i', 'd', **'est'**, 3)
    - 어휘사전: ['d', 'e', 'i', 'l', 'n', 'o', 'r', 's', 't', 'w', **'es'**, **'est'**, **'lo'**,**'low'**, **'low'**, **'ne'**, **'new'**, **'newest'**, **'wi'**, **'wid'**, **'widest'**]

-   위와 같이 어휘 사전이 만들어 지면 원래 어휘서전에 없던 것들에 대한 처리를 할 수있다.
    -   ex)
        -   'newer' :=> 'new', 'e', 'r',
        -   'lowest' :=> 'low', 'est'
        -   'wider' :=> 'wid', 'e', 'r'


# 데이터 조회

-   Korpora Dataset
    -   2017년 8월 부터 2019년 3월 까지 청와대 청원 게시판에 올라온 텍스트 말뭉치
    -   설치: `pip install korpora`


# Sentencepiece

-   구글에서 개발한 subword tokenizer 라이브러리로 BPE 인코딩 방식과 유사한 알고리즘을 이용해 토큰화하고 단어 사전을 생성한다.
-   설치: `pip install sentencepiece`


## Train

-   가지고 있는 말뭉치를 이용해 tokenzier를 만든다.
-   `SentencePieceTrainer.Train()`
-   Parameter
    -   input: 학습데이터 경로
    -   model_prefix: 학습된 모델 파일 저장이름
    -   vocab_size: 어휘 사전 크기
    -   model_type: 토크나이저 알고리즘 선택 - 'unigram', 'bpe', 'char', 'word'
    -   max_sentence_length: 최대 문장 길이

### 학습 결과파일

-   model_prefix지정파일명.model : 학습된 tokenizer
-   model_prefix지정파일명.vocab : 어휘사전이 저장된 파일 (text 파일)

> token에 `_` 가 붙은 것(under score가 아님. LOWER ONE EIGHTH BLOCK 문자로 unicode \u2581 이다.)
> sentencepiece는 공백을 토큰화 하는 과정에 `_` 로 표현한다. 그래서 'hello world'는 'hello_world'로 처리해 토큰화 한다.


## Tokenizer

-   `SentencePieceProcessor`
    -   SentencePieceTrainer.Train()으로 학습한 모델을 이용해 Tokenizer생성


# Wordpiece tokenizer

-   Byte Pair Encoding 이 빈도 기반이라면 wordpiece tokenizer는 확률 기반으로 글자 쌍을 병합한다.
-   두개 글자 쌍의 빈도수를 각 개별 글자 빈도수의 곱으로 나눈 점수가 가장 높은 순서대로 글자쌍을 묶어 나간다.

$$
score = \cfrac{f(x, y)}{f(x)\cdot f(y)}
$$

함수 f는 빈도를 나타내며 x, y는 병합하려는 하위 단어이다.

-   빈도사전: ('l','o','w', 5), ('l','o','w', 'e', 'r', 2), ('n', 'e', 'w', 'e', 's', 't', 6), ('w', 'i', 'd', 'e', 's', 't', 3)
-   어휘사전: ('d', 'e', 'i', 'l', 'n', 'o', 'r', 's', 't', 'w')
-   가장 빈도수가 높은 쌍은 'e','s'로 9번 등장한다. 이때 각 글자는 전체에서 각각 'e'는 17번, 's'는 9번 등장한다. 위 공식에 대입하면 score는 $\frac{9}{17 \times 9} \approx 0.06$ 이다.
-   'i'와 'd' 쌍은 3번만 등장하지만 전체에서 각각 'i' 3번, 'd' 3번 등장한다. 그래서 score는 $\frac{3}{3 \times 3} \approx 0.33$ 이다.
-   나타난 빈도수는 'es' 가 많치만 더 높은 score를 가지는 'id' 쌍을 병합한다.
-   빈도사전: ('l','o','w', 5), ('l','o','w', 'e', 'r', 2), ('n', 'e', 'w', 'e', 's', 't', 6), ('w', **'id'**, 'e', 's', 't', 3)
-   어휘사전: ('d', 'e', 'i', 'l', 'n', 'o', 'r', 's', 't', 'w', **'id'**)
    위의 작업을 반복해 연속된 글자 쌍이 더이상 나타나지 않거나 어휘 사전 max 크기에 도달할 때 까지 학습한다.


## 허깅페이스 Tokenizers 라이브러리 사용

-   Normalization(정규화), 사전토큰화(Pre-tokenization)을 제공
-   정규화
    -   일관된 형식으로 텍스트를 표준화하고, 모호함 제거를 위해 일부 문자 제거 또는 대체 작업을 한다.
-   사전 토큰화
    -   입력문장을 토큰화하기 전에 단어와 같은 작은 단위로 나누는 기능 제공.
-   설치: `pip instal tokenizers`
